# Parrallelization in Agentic Workflows
Wir nutzen Best Practices um unsere Agenten zu instanzieren.

In [11]:
import os
from openai import AzureOpenAI
from dotenv import load_dotenv

# Load environment variables and initialize OpenAI client
load_dotenv()

# Azure OpenAI Configuration - following the pattern you showed
api_key = os.getenv("AZURE_OPENAI_API_KEY")
azure_endpoint = os.getenv("AZURE_OPENAI_ENDPOINT")
azure_deployment = os.getenv("AZURE_OPENAI_DEPLOYMENT_NAME")
api_version = os.getenv("AZURE_OPENAI_API_VERSION", "2024-02-01")

# Model name should match your Azure deployment name
model = azure_deployment if azure_deployment else "gpt-4o-mini"

if not api_key or not azure_endpoint:
    raise ValueError("Azure OpenAI configuration missing. Please set AZURE_OPENAI_API_KEY, AZURE_OPENAI_ENDPOINT, and AZURE_OPENAI_DEPLOYMENT_NAME in your .env file.")

# Configure client for Azure OpenAI
client = AzureOpenAI(
    api_key=api_key,
    api_version=api_version,
    azure_endpoint=azure_endpoint,
)

# --- Helper Function for API Calls ---
def call_openai(system_prompt, user_prompt, model=model, temperature=0.0):
    """Simple wrapper for OpenAI API calls."""
    try:
        response = client.chat.completions.create(
            model=model,
            messages=[
                {"role": "system", "content": system_prompt},
                {"role": "user", "content": user_prompt}
            ],
            temperature=0.0
        )
        return response.choices[0].message.content
    except Exception as e:
        return f"An error occurred: {e}"
    

# Base Agent Classes
Wir erstellen eine BaseAgent-Klasse. Sie dient als Vorlage und stellt sicher, dass jeder Agent, den wir bauen, eine gemeinsame Struktur hat 
Die execute-Methode ist hier bewusst leer und wirft einen Fehler. Das ist ein Design-Pattern, das uns zwingt, für jeden spezialisierten Agenten, den wir davon ableiten, eine eigene, konkrete execute-Logik zu implementieren.


In [12]:
# agent_classes.py
class BaseAgent:
    """Eine Basisklasse, die die Grundstruktur für alle Agenten definiert."""
    def __init__(self, name: str, persona: str, client):
        self.name = name
        self.persona = persona
        self.client = client

    def execute(self, task: str, context: str = "") -> str:
        """Die Kernmethode, die von spezialisierten Agenten überschrieben wird."""
        raise NotImplementedError("Die 'execute'-Methode muss von einer Subklasse implementiert werden.")

# Spezialisierte Agent Classes
Alle Agent Klassen erben von der BaseAgent-Klasse. Das bedeutet, sie bekommt automatisch die __init__-Methode. 
Wir implementieren hier nur die execute-Methode und die tet_description method. Sie nimmt eine Aufgabe und optionalen Kontext entgegen, verpackt alles in einen System- und einen User-Prompt und ruft die OpenAI-API auf. 


In [21]:
# classes.py
class TechnologyAgent(BaseAgent):
    """Ein Agent, der auf die Recherche von Technologischen Trends spezialisiert ist."""

    def __init__(self, temperature=0.0):
        name = "Technology Expert"
        persona = "Du bist ein erfahrener Technologie-Analyst. Du recherchierst nur Informationen zu technologischen Trends"
        
        super().__init__(name, persona, client)
        self.temperature = temperature
        self.description = f"Dieser Agent heisst: {self.name.lower()}. Er kann recherchieren und Informationen zu technologischen Trends sammeln."

    def get_description(self):
        """Returns the agent's description."""
        return self.description

    def execute(self, task: str) -> str:
        print(f"INFO: {self.name} führt Recherche für '{task}' aus...")
        system_prompt = self.persona
        user_prompt = f"Recherchiere das folgende Thema aus deiner Perspektive: {task}"
        response = call_openai(
            model=model,
            system_prompt=system_prompt,
            user_prompt=user_prompt,
            temperature=self.temperature
        )
        return response


class MarketAnalysis(BaseAgent):
    """Ein Agent, der die Markt-Dynamiken recherchiert"""

    def __init__(self):
        name = "Market Analyst"
        persona = "Du bist ein erfahrener Markt-Analyst. Du recherchierst nur Informationen zu Marktdynamiken und wie sich ein Markt entwickelt"
        super().__init__(name, persona, client)
        self.description = f"Dieser Agent heisst: {self.name.lower()}. Er kann informationen zu Market Trends recherchieren"

    def get_description(self):
        """Returns the agent's description."""
        return self.description

    def execute(self, text: str) -> str:
        print(f"INFO: {self.name} übersetzt und fasst den Text zusammen...")
        system_prompt = self.persona
        user_prompt = f"Recherchiere das folgende Thema aus deiner Perspektive:\n\n{text}"
        response = call_openai(
            model=model,
            system_prompt=system_prompt,
            user_prompt=user_prompt
        )
        return response


class RegulationAgent(BaseAgent):
    """Ein Agent, der Regulationen in der Schweiz recherchiert"""

    def __init__(self):
        name = "Regulation Expert"
        persona = "Du bist ein erfahrener Regulationen-Spezialist. Du sammelst nur Informationen zu Regulationen zu einem Thema in der Schweiz."
        super().__init__(name, persona, client)
        self.description = f"Dieser Agent heisst: {self.name.lower()}. Er kann Informationen zu aktuellen Schweizer Regulationen sammeln."

    def get_description(self):
        """Returns the agent's description."""
        return self.description

    def execute(self, text: str) -> str:
        print(f"INFO: {self.name} überprüft die Fakten...")
        system_prompt = self.persona
        user_prompt = f"Recherchiere zum folgenden Thema aus deiner Perspektive:\n\n{text}"
        response = call_openai(
            model=model,
            system_prompt=system_prompt,
            user_prompt=user_prompt
        )
        return response


class SummaryAgent:
    def __init__(self):
        self.client = client

    def run(self, text, inputs):
        combined_prompt = (
            f"The user asked: '{text}'\n\n"
            f"Here are the expert responses:\n"
            f"- Policy Expert: {inputs['policy']}\n\n"
            f"- Technology Expert: {inputs['tech']}\n\n"
            f"- Market Expert: {inputs['market']}\n\n"
            "Please summarize the combined insights into a single clear and concise response."
        )
        print(f"Summary Agent resolving prompt: {combined_prompt}")

        response = self.client.chat.completions.create(
            model=model,
            messages=[
                {"role": "system", "content": "You are an energy strategist skilled at synthesizing expert insights."},
                {"role": "user", "content": combined_prompt}
            ],
            temperature=0.7,
        )
        return response.choices[0].message.content

# Threading das Herzstück

In [16]:
import threading

agent_outputs = {}

# The shared user prompt
user_prompt = "What are current trends shaping the future of the energy industry?"

In [22]:
def main():
           # Initialize agents - corrected parameters
           policy_agent = RegulationAgent()
           tech_agent = TechnologyAgent(temperature=2.0)
           market_agent = MarketAnalysis()
           summary_agent = SummaryAgent()

           # Funktion um alle Agenten zu starten und unter dem jeweiligen Key die ergebnisse zu speichern.
           def run_agent(agent, key):
               agent_outputs[key] = agent.execute(user_prompt)

           # Hier erstellen wir die Threads für die Parallele Execution
           threads = [
               threading.Thread(target=run_agent, args=(policy_agent, "policy")),
               threading.Thread(target=run_agent, args=(tech_agent, "tech")),
               threading.Thread(target=run_agent, args=(market_agent, "market"))
           ]

           # Start all threads gleichtzeitig
           for t in threads:
               t.start()

           # Wait for all threads zum fertigstellen
           for t in threads:
               t.join()

           # Starte den Zusammenfassungs Agent um auf Basis aller Ergebnisse in den Threads den Report zu schreiben.
           final_summary = summary_agent.run(user_prompt, agent_outputs)

           print("\n=== FINAL SUMMARY ===\n")
           print(final_summary)

if __name__ == "__main__":
    main()



INFO: Regulation Expert überprüft die Fakten...
INFO: Technology Expert führt Recherche für 'What are current trends shaping the future of the energy industry?' aus...
INFO: Market Analyst übersetzt und fasst den Text zusammen...
Summary Agent resolving prompt: The user asked: 'What are current trends shaping the future of the energy industry?'

Here are the expert responses:
- Policy Expert: In der Schweiz beeinflussen verschiedene regulatorische Trends die zukünftige Entwicklung der Energiebranche. Hier eine Übersicht der wichtigsten aktuellen regulatorischen Entwicklungen und Trends:

1. **Energiegesetz (EnG) und Energiestrategie 2050**  
   Die Energiestrategie 2050 der Schweiz zielt auf eine nachhaltige Energieversorgung ab, mit Fokus auf Energieeffizienz, Ausbau erneuerbarer Energien und Reduktion fossiler Brennstoffe. Das revidierte Energiegesetz (EnG), das seit 2018 in Kraft ist, fördert den Ausbau von Solar-, Wind- und Wasserkraft sowie die Verbesserung der Energieeffizienz in